In [1]:
# Assignment 13 by Sreedhara Jagatagar  Sreenivasa
#Read the following data set: https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data
#Task:

#Import Required libraray, modules and Packages
import pandas as pd
from numpy import genfromtxt
from time import time
from datetime import datetime
from sqlalchemy import Column, Integer, Float, Date, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import func
from sqlalchemy import Table

#creata a data frame just to see the data
adult_Data_df=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', header=None,
                          names=['Age','WorkClass','fnlwgt','Education','Education_Num','Marital_Status','Occupation',
                                 'Felationship','Race','Sex','Capital_Gain','Capital_Loss','Hours_per_week','Native_Country',
                                 'Income_Range'])
#Print the Data Field
adult_Data_df.head(5)


,Age,WorkClass,fnlwgt,Education,Education_Num,Marital_Status,Occupation,Felationship,Race,Sex,Capital_Gain,Capital_Loss,Hours_per_week,Native_Country,Income_Range
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
# Assignment 13 by Sreedhara Jagatagar  Sreenivasa
#1. Create an sqlalchemy engine using a sample from the data set

#decliare base
Base = declarative_base()

#drop table table sqlalchemy database engine if already exists
try:
    sql = 'DROP TABLE IF EXISTS adult_data;'
    result = engine.execute(sql)  
    print('DB is Dropped')    
except Exception as e:
    print ('Error',e)

#Function to load Data to sqlalchemy database engine
def Load_Data(file_name):
    #load data and return the data list
    data = genfromtxt(file_name, delimiter=',', skip_header=0,dtype="i8,S50,i8,S50,i8,S50,S50,S50,S50,S50,f8,f8,i8,S50,S50",
                      replace_space='_')
    return data.tolist()

#Define SQLAlchemy adult Tables using above data file
class adult_data(Base):
    #Tell SQLAlchemy what the table name is and if there's any table-specific arguments it should know about
    __tablename__ = 'adult_data'
    __table_args__ = {'sqlite_autoincrement': True}
    
    #tell SQLAlchemy the name of column and its attributes:
    id = Column(Integer, primary_key=True, nullable=False) 
    Age = Column(Integer)
    WorkClass = Column(String)
    fnlwgt = Column(Integer)
    Education = Column(String)
    Education_Num = Column(Integer)
    Marital_Status = Column(String)
    Occupation = Column(String)
    Relationship = Column(String)
    Race = Column(String)
    Sex = Column(String)
    Capital_Gain = Column(Float)
    Capital_Loss = Column(Float)
    Hours_per_week = Column(Integer)
    ative_Country = Column(String)
    Income_Range = Column(String)
    
#Main Code
if __name__ == "__main__":
    t = time()
    try:

        #Create the database
        engine = create_engine('sqlite:///csv_test.db')
        Base.metadata.create_all(engine)

        #Create the session
        session = sessionmaker()
        session.configure(bind=engine)
        s = session()
        print('sqlalchemy DB engine and DB is created ')        
    #Capture Exception If any
    except Exception as e:
        print ('Session ',e)
    
    
    j=0
    try:
        #store File Name and Call Load_Data function 
        file_name = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data" #sample CSV file used:  http://www.google.com/finance/historical?q=NYSE%3AT&ei=W4ikVam8LYWjmAGjhoHACw&output=csv
        data = Load_Data(file_name) 
        
        print('Data is loaded from adult.data file')

        #Loop Through and insert record to adult_data tables.
        for i in data:
            j=j+1
            record = adult_data(**{
                'id' : j,
                'Age' : i[0],
                'WorkClass' : i[1],
                'fnlwgt' : i[2],
                'Education' : i[3],
                'Education_Num' : i[4],
                'Marital_Status' : i[5],
                'Occupation' : i[6],
                'Relationship': i[7],
                'Race' : i[8],
                'Sex' : i[9].strip(),
                'Capital_Gain' : i[10],
                'Capital_Loss' : i[11],
                'Hours_per_week' : i[12],
                'ative_Country' : i[13],
                'Income_Range' : i[14]
            })
            #print(record.fnlwgt)
            s.add(record) #Add all the records
        print('Data is inserted into adult_data table at SQLAlchemy')
            
        #Commit the data
        s.commit() #Attempt to commit all the records
        print('Commited adult data at SQLAlchemy DB')
        
        #Fetch and Print the count of data insert in adult_data
        our_adult_data_count = s.query(adult_data).count()
        print('Data Count (Inserted Rows from Adult.data file at SQLAlchemy) is: ', our_adult_data_count)

    except Exception as e:
        print('Error2 ',e)
        s.rollback() #Rollback the changes on error
    finally:
        #s.close() #Close the connection
        print ("Time elapsed: " + str(time() - t))#0.091s)

DB is Dropped
sqlalchemy DB engine and DB is created 
Data is loaded from adult.data file
Data is inserted into adult_data table at SQLAlchemy
Commited adult data at SQLAlchemy DB
Data Count (Inserted Rows from Adult.data file at SQLAlchemy) is:  32561
Time elapsed: 6.8022496700286865


In [5]:
# Assignment 13 by Sreedhara Jagatagar  Sreenivasa
#2. Write two basic update queries

print('First Update queries start')
print(' ')

#Get the count of rows which are having age 25 years before the update
Update_count=(s.query(adult_data).filter(adult_data.Age==25)).count()
print('Number of Records which are having Age 25 years before the update',Update_count)

#Update the age with with addtional 150 years to identify the rows
s.query(adult_data).filter(adult_data.Age==25).\
    update({adult_data.Age: adult_data.Age+150}, synchronize_session=False)

#Get the count of rows which are having more than 150 years
Update_count=(s.query(adult_data).filter(adult_data.Age>150)).count()
print('Number of Records which are having Age more than 150  years after update',Update_count)

#Get the count of rows which are having 25 years after the update
Update_count=(s.query(adult_data).filter(adult_data.Age==25)).count()
print('Number of Records which are having Age 25 years after update',Update_count)


print(' ')
print('First Update queries End')
print(' ')

print('Second Update queries start')
print(' ')

#Get the Male count before the update
Update_count=(s.query(adult_data).filter(adult_data.Sex==b'Male')).count()
print('Count of Rows which are having sex "Male" before update  ',Update_count)

#Update the sex with "Male-Updated"
s.query(adult_data).filter(adult_data.Sex==b'Male').\
    update({adult_data.Sex: 'Male-Updated'}, synchronize_session=False)

#Get the count of rows which are having sex "Male_updated" after update
Update_count=(s.query(adult_data).filter(adult_data.Sex=='Male-Updated')).count()
print('Count of Rows which are updated with "Male-Updated"  after Update ',Update_count)

#Get the Rows which are having Sex "Male" string
Update_count=(s.query(adult_data).filter(adult_data.Sex==b'Male')).count()
print('Count of Rows which are having sex "Male" After the update  ',Update_count)

print(' ')
print('Second Update queries End')

#commit
s.commit()

First Update queries start
 
Number of Records which are having Age 25 years before the update 841
Number of Records which are having Age more than 150  years after update 841
Number of Records which are having Age 25 years after update 0
 
First Update queries End
 
Second Update queries start
 
Count of Rows which are having sex "Male" before update   21790
Count of Rows which are updated with "Male-Updated"  after Update  21790
Count of Rows which are having sex "Male" After the update   0
 
Second Update queries End


In [6]:
# Assignment 13 by Sreedhara Jagatagar  Sreenivasa
#3. Write two delete queries

#Build the query - 1st Delete Query
adult_table = Table('adult_data', adult_data.metadata, autoload=True)
adult_addresses = s.query(adult_table).filter_by(fnlwgt=215646)

print('Data Selection Query to Delete - First Query')
print(' ')
print(adult_addresses)
print(' ')

#Number of records Before Deltion 
before_count=s.query(adult_data).count()

#Execute the delete statement
adult_addresses.delete(synchronize_session=False)

#Number of records after Deletion 
after_count=s.query(adult_data).count()

#Print row counts
print('Rows Count of Adult Data set before deletion :',before_count)
print('Rows Count of Adult Data set after deletion :',after_count)
print('Number of Rows Deleted : ', before_count-after_count)

#commit the data
s.commit()

Data Selection Query to Delete - First Query
 
SELECT adult_data.id AS adult_data_id, adult_data."Age" AS "adult_data_Age", adult_data."WorkClass" AS "adult_data_WorkClass", adult_data.fnlwgt AS adult_data_fnlwgt, adult_data."Education" AS "adult_data_Education", adult_data."Education_Num" AS "adult_data_Education_Num", adult_data."Marital_Status" AS "adult_data_Marital_Status", adult_data."Occupation" AS "adult_data_Occupation", adult_data."Relationship" AS "adult_data_Relationship", adult_data."Race" AS "adult_data_Race", adult_data."Sex" AS "adult_data_Sex", adult_data."Capital_Gain" AS "adult_data_Capital_Gain", adult_data."Capital_Loss" AS "adult_data_Capital_Loss", adult_data."Hours_per_week" AS "adult_data_Hours_per_week", adult_data."ative_Country" AS "adult_data_ative_Country", adult_data."Income_Range" AS "adult_data_Income_Range" 
FROM adult_data 
WHERE adult_data.fnlwgt = ?
 
Rows Count of Adult Data set before deletion : 32561
Rows Count of Adult Data set after deletion : 

In [7]:
#Build the query 2nd Delete Query
adult_table = Table('adult_data', adult_data.metadata, autoload=True)
adult_addresses = s.query(adult_table).filter_by(Age=50)

print('Data Selection Query to Delete - Second Query')
print(' ')
print(adult_addresses)
print(' ')

#Number of records Before Deltion 
before_count=s.query(adult_data).count()

#Execute Deleate statement
adult_addresses.delete(synchronize_session=False)

#Number of records after Deltion 
after_count=s.query(adult_data).count()

#Calcualate and print the count
print('Rows Count of Adult Data set before deletion :',before_count)
print('Rows Count of Adult Data set after deletion :',after_count)
print('Number of Rows Deleted : ', before_count-after_count)

#commit the data
s.commit()

Data Selection Query to Delete - Second Query
 
SELECT adult_data.id AS adult_data_id, adult_data."Age" AS "adult_data_Age", adult_data."WorkClass" AS "adult_data_WorkClass", adult_data.fnlwgt AS adult_data_fnlwgt, adult_data."Education" AS "adult_data_Education", adult_data."Education_Num" AS "adult_data_Education_Num", adult_data."Marital_Status" AS "adult_data_Marital_Status", adult_data."Occupation" AS "adult_data_Occupation", adult_data."Relationship" AS "adult_data_Relationship", adult_data."Race" AS "adult_data_Race", adult_data."Sex" AS "adult_data_Sex", adult_data."Capital_Gain" AS "adult_data_Capital_Gain", adult_data."Capital_Loss" AS "adult_data_Capital_Loss", adult_data."Hours_per_week" AS "adult_data_Hours_per_week", adult_data."ative_Country" AS "adult_data_ative_Country", adult_data."Income_Range" AS "adult_data_Income_Range" 
FROM adult_data 
WHERE adult_data."Age" = ?
 
Rows Count of Adult Data set before deletion : 32560
Rows Count of Adult Data set after deletion : 

In [8]:
# Assignment 13 by Sreedhara Jagatagar  Sreenivasa
#4. Write two filter queries

# Count of data set
print('Rows Count of Adult Data set :',s.query(adult_data).count())

print(' ')
print('1st Filter Query')
print(' ')

#Build the Filter query
our_adult_data = (s.query(adult_data)).filter_by(fnlwgt=77516)

#Print the filter query
print(our_adult_data)

#Execute the Filter query
a=s.execute(our_adult_data)

print(' ')
print('Fetched Data from filter query ')
print(' ')

#print the Fetched data
print(a.fetchall())

Rows Count of Adult Data set : 31958
 
1st Filter Query
 
SELECT adult_data.id AS adult_data_id, adult_data."Age" AS "adult_data_Age", adult_data."WorkClass" AS "adult_data_WorkClass", adult_data.fnlwgt AS adult_data_fnlwgt, adult_data."Education" AS "adult_data_Education", adult_data."Education_Num" AS "adult_data_Education_Num", adult_data."Marital_Status" AS "adult_data_Marital_Status", adult_data."Occupation" AS "adult_data_Occupation", adult_data."Relationship" AS "adult_data_Relationship", adult_data."Race" AS "adult_data_Race", adult_data."Sex" AS "adult_data_Sex", adult_data."Capital_Gain" AS "adult_data_Capital_Gain", adult_data."Capital_Loss" AS "adult_data_Capital_Loss", adult_data."Hours_per_week" AS "adult_data_Hours_per_week", adult_data."ative_Country" AS "adult_data_ative_Country", adult_data."Income_Range" AS "adult_data_Income_Range" 
FROM adult_data 
WHERE adult_data.fnlwgt = ?
 
Fetched Data from filter query 
 
[(1, 39, b' State-gov', 77516, b' Bachelors', 13, b' N

In [9]:
#Second filter queries
print('2nd Filter Query')
print(' ')

#Build the query
our_adult_data = (s.query(adult_data)).filter_by(Age=80)

#Print the build query
print(our_adult_data)

#execute the query
a=s.execute(our_adult_data)

print(' ')
print('Fetched Data from 2nd Filte quries ')
print(' ')

#print the data fetched
print(a.fetchone())

2nd Filter Query
 
SELECT adult_data.id AS adult_data_id, adult_data."Age" AS "adult_data_Age", adult_data."WorkClass" AS "adult_data_WorkClass", adult_data.fnlwgt AS adult_data_fnlwgt, adult_data."Education" AS "adult_data_Education", adult_data."Education_Num" AS "adult_data_Education_Num", adult_data."Marital_Status" AS "adult_data_Marital_Status", adult_data."Occupation" AS "adult_data_Occupation", adult_data."Relationship" AS "adult_data_Relationship", adult_data."Race" AS "adult_data_Race", adult_data."Sex" AS "adult_data_Sex", adult_data."Capital_Gain" AS "adult_data_Capital_Gain", adult_data."Capital_Loss" AS "adult_data_Capital_Loss", adult_data."Hours_per_week" AS "adult_data_Hours_per_week", adult_data."ative_Country" AS "adult_data_ative_Country", adult_data."Income_Range" AS "adult_data_Income_Range" 
FROM adult_data 
WHERE adult_data."Age" = ?
 
Fetched Data from 2nd Filte quries 
 
(431, 80, b' ?', 107762, b' HS-grad', 9, b' Widowed', b' ?', b' Not-in-family', b' White',

In [10]:
# Assignment 13 by Sreedhara Jagatagar  Sreenivasa
#5. Write two function queries

print("First Function Queries")
print(' ')

#Build the query
adult_count_query = s.query(func.count(adult_data.id).label('total'))
print(adult_count_query)

#Get the count
adult_count_result=s.execute(adult_count_query)
count= adult_count_result.fetchall()

count_of_id = str(count).replace('[(','').replace(',)]','')

print(' ')
print('Total number of Adult Data ID is :',count_of_id)
print(' ')
print("First Function Queries End ")
print(' ')

print("Second Function Queries")
print(' ')

#Build the query
adult_sum_query = s.query(func.sum(adult_data.Capital_Gain).label('Capital Gains'))
print(adult_sum_query)

#Get the count
adult_sum_result=s.execute(adult_sum_query)
sum= adult_sum_result.fetchall()

Sum_captial_gains = str(sum).replace('[(','').replace(',)]','')

print(' ')
print('Total Amount of Capital Gain is : ',Sum_captial_gains)
print(' ')
print("Second Function Queries End ")
print(' ')

First Function Queries
 
SELECT count(adult_data.id) AS total 
FROM adult_data
 
Total number of Adult Data ID is : 31958
 
First Function Queries End 
 
Second Function Queries
 
SELECT sum(adult_data."Capital_Gain") AS "Capital Gains" 
FROM adult_data
 
Total Amount of Capital Gain is :  33622455.0
 
Second Function Queries End 
 
